Download the List_of_postal_codes_of_Canada to an html file:

In [3]:
!curl https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M > postal_codes.html
print('Data downloaded!')

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 78670  100 78670    0     0  44122      0  0:00:01  0:00:01 --:--:-- 44122
Data downloaded!


parse the html file using BeautifulSoup:

In [15]:
from bs4 import BeautifulSoup

with open('postal_codes.html') as html_file:
    html_data = html_file.read()

    soup=BeautifulSoup(html_data,"lxml")
    #print(soup.prettify())
    result = soup.find("table")
    

<class 'str'>


In [18]:
result

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North

Transform the parse result to an object list(with properties: Postcode, Borough & Neighbourhood)

In [64]:
data_list = []
for idx, tr in enumerate(result.find_all('tr')):
    if idx != 0:
        tds = tr.find_all('td')
        data_list.append({
            'PostalCode': tds[0].contents[0],
            'Borough': tds[1].contents[0],
            'Neighbourhood': tds[2].contents[0],
        })
print(data_list)

[{'PostalCode': 'M1A', 'Borough': 'Not assigned', 'Neighbourhood': 'Not assigned\n'}, {'PostalCode': 'M2A', 'Borough': 'Not assigned', 'Neighbourhood': 'Not assigned\n'}, {'PostalCode': 'M3A', 'Borough': <a href="/wiki/North_York" title="North York">North York</a>, 'Neighbourhood': <a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>}, {'PostalCode': 'M4A', 'Borough': <a href="/wiki/North_York" title="North York">North York</a>, 'Neighbourhood': <a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>}, {'PostalCode': 'M5A', 'Borough': <a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a>, 'Neighbourhood': <a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>}, {'PostalCode': 'M6A', 'Borough': <a href="/wiki/North_York" title="North York">North York</a>, 'Neighbourhood': <a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>}, {'PostalCode': 'M6A', 'Borough': <a href="/wiki/North_York" title="North Y

create the data_frame:

In [65]:
import pandas as pd
data_frame = pd.DataFrame(data_list)

In [66]:
data_frame.head(10)

,PostalCode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,[North York],[Parkwoods]
3,M4A,[North York],[Victoria Village]
4,M5A,[Downtown Toronto],[Harbourfront]
5,M6A,[North York],[Lawrence Heights]
6,M6A,[North York],[Lawrence Manor]
7,M7A,[Queen's Park],Not assigned\n
8,M8A,Not assigned,Not assigned\n
9,M9A,[Downtown Toronto],[Queen's Park]


Define a function to process the Borough & Neighbourhood data:
    1. clear all the '\n's in the text;
    2. extract title from the <a href structrue;
    3. return the title.

In [67]:
def clean_data(x):
    result = str(x)
    result = result.replace('\n','')
    if('Not assigned' in result):
        return result
    if('href' not in result):
        return result
    #print(x)
    return re.findall('title="(.+?)"', result)[0]

Apply the function to 'Borough' and 'Neighbourhood' columns, and drop all rows where borough is Not assigned:

In [68]:
data_frame['Neighbourhood'] = data_frame['Neighbourhood'].apply(lambda x:clean_data(x))
data_frame['Borough'] = data_frame['Borough'].apply(lambda x:clean_data(x))

data_frame = data_frame[data_frame['Borough']!='Not assigned'].reset_index(drop=True)

In [69]:
data_frame.head(10)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Regent Park
3,M6A,North York,Lawrence Heights
4,M6A,North York,Lawrence Manor
5,M7A,Queen's Park (Toronto),Not assigned
6,M9A,Downtown Toronto,Queen's Park (Toronto)
7,M1B,"Scarborough, Toronto","Rouge, Toronto"
8,M1B,"Scarborough, Toronto","Malvern, Toronto"
9,M3B,North York,Don Mills North


Finally we get the postalcode data:

In [70]:
data_frame.shape

(210, 3)